# Afterpulse Detection (dead-time / internal reflections)
To figure out how to detect and remove afterpulses, and plot examples.
- use photon rate just around the lake surface elevation (depending on strong/weak beam??)
- then use peaks distances that are commonly found

### From the literature
afterpulses at: ∼0.45, ∼0.9, ∼2.3, and ∼4.2 m

The afterpulses captured from on-orbit measurements are caused by three different reasons: (1) the effects of the dead-time circuit (∼3 ns) due to PMT saturation; (2) the effects of optical reflections within the ATLAS receiver optical components; (3) PMT afterpulses. The echoes separated by ∼0.45 m are attributed to the effect of the dead-time circuit (∼3 ns) due to PMT saturation. The echoes at ∼2.3 and ∼4.2 m below the primary surface returns are caused by the optical reflections within the ATLAS receiver optical components, while the echoes from ∼10 to ∼45 m away from the primary surface signal are due to the PMT afterpulses with a longer time delay.

Lu, X., Hu, Y., Yang, Y., Vaughan, M., Palm, S., Trepte, C., ... & Baize, R. (2021). Enabling value added scientific applications of ICESat‐2 data with effective removal of afterpulses. Earth and Space Science, 8(6), e2021EA001729.

In [1]:
%matplotlib widget
import pickle
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import Image, display
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.signal import find_peaks
plt.rcParams.update({'font.size': 8})

In [2]:
# lake 3 (gt1l, weak)
# lake 13* (gt1r, strong)
# lake 23 (gt1r, strong)
# lake 24 (gt1r, strong)
# lake 28 (gt1r, strong)
# lake 31** (gt2l, weak)
# lake 35** (gt2l, weak)
# lake 36** (gt2l, weak)
# lake 37** (gt2l, weak)
# lake 39*** (gt2l, weak)
# lake 43 (gt2r, strong)
# lake 44* (gt2r, strong)
# lake 45 (gt2r, strong)
# lake 46** (gt2r, strong)
# lake 48 (gt2r, strong)
# lake 50*** (gt2r, strong)
# lake 51** (gt2r, strong)
# lake 53** (gt2r, strong)
# lake 54 (gt2r, strong)

In [10]:
thresh_upper = 1.0
thresh_lower = -5.0
bin_h=0.01
smooth_h=0.1
extent_buffer = 20.0
smooth_pulse = 11
strength = 'strong'

cols_pk = ['black','#CD104D', '#E14D2A', '#FD841F', '#8FE3CF', '#256D85']
lsty_pk = ['-', '-', '--', ':', '-', ':']
elev_pk_lake = [0.0, -0.55, -0.845, -1.36, -2.4, -4.2]
elev_pk_puls = [0.0, -0.55, -0.91, -1.465, -2.4, -4.2]
widths_pk = [0.225, 0.225, 0.225, 0.225, 0.3, 0.3] # 0.225 m on each side makes it 0.45 total, which is the dead-time for ATLAS
df_pks_info = pd.DataFrame({'h_lake': elev_pk_lake, 'h_pulse': elev_pk_puls, 'width': widths_pk, 'color': cols_pk, 'ls': lsty_pk})

def group_by_pulse(df_in, smoothing, beam_strength):
    thegroup = df_in.groupby('pulseid')
    df_grouped = thegroup[['xatc', 'h']].mean()
    norm_factor = 4 if beam_strength == 'weak' else 16
    df_grouped['ph_count'] = thegroup['h'].count() / norm_factor
    df_grouped['ph_count_smooth'] = np.array(df_grouped.ph_count.rolling(smoothing,center=True,min_periods=1).mean())
    return df_grouped

## get peak elevations from lake data

In [11]:
plt.close('all')
lake_idxs = [3, 13, 23, 24, 28, 31, 35, 36, 37, 39, 43, 44, 45, 46, 48, 50, 51, 53, 54]
# lake_idxs = [50]

df_list = []
# for i in range(71):
for i in lake_idxs:
    
    fn = 'pickles/specular%02i.pkl' % i
    with open(fn, 'rb') as f:
        lk = pickle.load(f)
        
    surf_elev = lk['surface_elevation']
    df = lk['photon_data']
    beam_strength = lk['beam_strength']
    dfs = df[(df.h < (thresh_upper+surf_elev)) & (df.h > (thresh_lower+surf_elev))].copy()
    dfs['h'] = dfs.h - surf_elev
    dfs['pulseid'] = dfs.apply(lambda row: 1000*row.mframe+row.ph_id_pulse, axis=1)
    dfs = dfs.set_index('pulseid')
    df_surf_photons = dfs[(dfs.h >= -0.25) & (dfs.h < 0.25)].copy()
    df_pulses = group_by_pulse(df_surf_photons, smooth_pulse, beam_strength)
    df_join = dfs.join(df_pulses, how='left', rsuffix='_pulse')
    df_join['h_relative_to_saturated_peak'] = df_join.h - df_join.h_pulse
    if (strength not in ['weak','strong']) | (strength == beam_strength):
        df_list.append(df_join)
    
dfs = pd.concat(df_list)
df_saturated = dfs[dfs.ph_count_smooth > 0.93]
# df_saturated = dfs[dfs.ph_count > 0.99]

# histogram binning for afterpulse peaks vs. elevation
bins = np.arange(start=thresh_lower, stop=thresh_upper, step=bin_h)
mids = bins[:-1] + 0.5 * bin_h
smooth = int(smooth_h/bin_h)
if smooth %2 == 0: smooth += 1
def get_histograms(ph_heights):
    hist_h = np.histogram(ph_heights, bins=bins)
    hist_h_smooth = np.array(pd.Series(hist_h[0]).rolling(smooth,center=True,min_periods=1).mean())
    hist_h_plot = np.array(hist_h[0]) / hist_h_smooth.max()
    hist_h_smooth /= hist_h_smooth.max()
    return hist_h_plot, hist_h_smooth

hist_h_all, hist_h_smooth_all = get_histograms(dfs.h)
hist_h_sat, hist_h_smooth_sat = get_histograms(df_saturated.h)
hist_h_sat_adjusted, hist_h_smooth_sat_adjusted = get_histograms(df_saturated.h_relative_to_saturated_peak)

fig, (ax,ax2) = plt.subplots(ncols=2, figsize=[9, 5], dpi=100)
ylms = (thresh_lower, thresh_upper)
xlim_ax2 = (1e-4,10)

# histogram showing peaks for specular returns
ax.scatter(hist_h_all, mids, s=3, color='black', lw=0.5, edgecolors='none', alpha=0.15)
ax.plot(hist_h_smooth_all, mids, 'k-', lw=1)
ax.set_xlabel('normalized photon counts')
ax.set_ylabel('elevation relative to lake surface')
ax.set_xlim(xlim_ax2)
ax.set_ylim(ylms)
ax.set_xscale('log')
for i in range(len(df_pks_info)):
    thispk = df_pks_info.iloc[i]
    thispeak_height = hist_h_smooth_all[np.argmin(np.abs(mids-thispk.h_lake))]
    ax.plot([xlim_ax2[0], thispeak_height], [thispk.h_lake]*2, color=thispk.color, ls=thispk.ls, zorder=-1000)
    if i == 0:
        ax.text(1.1*xlim_ax2[0], thispk.h_lake, 'lake surface', color=thispk.color, ha='left', va='bottom')
    else:
        ax.text(thispeak_height, thispk.h_lake, '    %.2f m' % thispk.h_lake, color=thispk.color, weight='bold', va='center')


ax2.scatter(hist_h_sat_adjusted, mids, s=3, color='black', lw=0.5, edgecolors='none', alpha=0.15)
ax2.plot(hist_h_smooth_sat_adjusted, mids, 'k-', lw=1)
ax2.set_xlabel('normalized photon counts (saturated pulses only)')
ax2.set_ylabel('elevation relative to saturated surface return')
ax2.set_xlim(xlim_ax2)
ax2.set_ylim(ylms)
ax2.set_xscale('log')
for i in range(len(df_pks_info)):
    thispk = df_pks_info.iloc[i]
    thispeak_height = hist_h_smooth_sat_adjusted[np.argmin(np.abs(mids-thispk.h_pulse))]
    ax2.plot([xlim_ax2[0], thispeak_height], [thispk.h_pulse]*2, color=thispk.color, ls=thispk.ls, zorder=-1000)
    if i == 0:
        ax2.text(1.1*xlim_ax2[0], thispk.h_pulse, 'saturated surface return', color=thispk.color, ha='left', va='bottom')
    else:
        ax2.text(thispeak_height, thispk.h_pulse, '    %.2f m' % thispk.h_pulse, color=thispk.color, weight='bold', va='center')
        
fig.suptitle('ICESat-2 afterpulses over melt lakes', y=0.98, fontsize=10)
fig.tight_layout()

figname = 'figs_afterpulses/ICESat-2-afterpulses-melt-lakes-strong-beams.jpg'
fig.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.close('all')
i = 50
fn = 'pickles/specular%02i.pkl' % i
with open(fn, 'rb') as f:
    lk = pickle.load(f)

if smooth_pulse%2 == 0: smooth_pulse += 1

surf_elev = lk['surface_elevation']
surf_ext = lk['surface_extent_detection']
df = lk['photon_data']
beam_strength = lk['beam_strength']
dfs = df[(df.h < (thresh_upper+surf_elev)) & (df.h > (thresh_lower+surf_elev))].copy()
dfs['h'] = dfs.h - surf_elev
is_in_extent = np.full(len(dfs), False, dtype=np.bool_)
for ext in surf_ext:
    is_in_extent[(dfs.xatc > (ext[0]-extent_buffer)) & (dfs.xatc < (ext[1]+extent_buffer))] = True
dfs = dfs[is_in_extent].copy()

# histogram binning for afterpulse peaks vs. elevation
bins = np.arange(start=thresh_lower, stop=thresh_upper, step=bin_h)
mids = bins[:-1] + 0.5 * bin_h
hist_h = np.histogram(dfs.h, bins=bins)
smooth = int(smooth_h/bin_h)
if smooth %2 == 0: smooth += 1
hist_h_smooth = np.array(pd.Series(hist_h[0]).rolling(smooth,center=True,min_periods=1).mean())
hist_h_plot = np.array(hist_h[0]) / hist_h_smooth.max()
hist_h_smooth /= hist_h_smooth.max()
# peaks, peak_props = find_peaks(hist_h_smooth, height=0.1, distance=int(0.4/bin_h_spec), prominence=0.1)

# add a unique id for each pulse to group by, then look for per-pulse photon counts
dfs['pulseid'] = dfs.apply(lambda row: 1000*row.mframe+row.ph_id_pulse, axis=1)
pulse_dfs = []
for i in range(len(df_pks_info)):
    pkinfo = df_pks_info.iloc[i]
    photon_df = dfs[(dfs.h >= (pkinfo.h_lake - pkinfo.width)) & (dfs.h < (pkinfo.h_lake + pkinfo.width))]
    pulse_dfs.append(group_by_pulse(photon_df,smooth_pulse,beam_strength))

#__________________________________________________________________
# make the figure
fig, ((ax,ax2),(ax3,ax4)) = plt.subplots(nrows=2, ncols=2, figsize=[9, 5], dpi=100)
xlms = (surf_ext[0][0], surf_ext[-1][1])
ylms = (thresh_lower, thresh_upper)

#-------------------------------------------------
# selected photons
pscatt = ax.scatter(dfs.xatc, dfs.h, s=15, c='k', alpha=0.1, edgecolors='none',label='ATL03 data')
ax.scatter(dfs.xatc, dfs.h, s=0.1, c='r', alpha=0.2, edgecolors='none')

# title and labels
ax.set_xlabel('along-track distance [m]')
ax.set_ylabel('elevation from lake surface [m]')
ax.set_xlim(xlms)
ax.set_ylim(ylms)

#-------------------------------------------------
# histogram showing peaks for specular returns
xlim_ax2 = (1e-4,10)
ax2.scatter(hist_h_plot, mids, s=3, color=(0.75, 0.75, 0.75), edgecolors='none')
ax2.plot(hist_h_smooth, mids, 'k-', lw=1)
# ax2.plot(xlim_ax2, [0]*2, 'b-', lw=0.5)
ax2.set_xlabel('normalized photon counts')
ax2.set_xlim(xlim_ax2)
ax2.set_ylim(ylms)
ax2.set_xscale('log')

for i in range(len(df_pks_info)):
    thispk = df_pks_info.iloc[i]
    thispeak_height = hist_h_smooth[np.argmin(np.abs(mids-thispk.h_lake))]
    ax2.plot([xlim_ax2[0], thispeak_height], [thispk.h_lake]*2, color=thispk.color, ls=thispk.ls, zorder=-1000)
# ax2.axes.yaxis.set_ticklabels([])

#-------------------------------------------------
# plot photon counts per shot
alph = 0.1
thisax = ax3
def plot_counts(df_in, df_pks_info):
    thisax.scatter(df_in.xatc, df_in.ph_count, s=3, color=df_pks_info.iloc[i].color, edgecolors='none', alpha=alph)
    thisax.plot(df_in.xatc, df_in.ph_count_smooth, color=df_pks_info.iloc[i].color, ls=df_pks_info.iloc[i].ls)

for i,this_peak_pulse_df in enumerate(pulse_dfs):
    plot_counts(this_peak_pulse_df, df_pks_info)
    
ax3.set_xlim(xlms)
ax3.set_yscale('log')
         
#-------------------------------------------------
# the full lake, for reference
ax4.scatter(df.xatc, df.h, s=6, c=df.snr, alpha=0.3, edgecolors='none', cmap=cmc.lajolla, vmin=0, vmax=1)
rng = np.abs(surf_elev-np.min(lk['detection_2nd_returns']['h']))
ax4.set_xlim((df.xatc.min(),df.xatc.max()))
ax4.set_ylim((surf_elev-2*rng, surf_elev+rng))

fig.suptitle('example of specular return afterpulses over a melt lake in ICESat-2 data', y=0.98, fontsize=10)
fig.tight_layout()

figname = 'figs_afterpulses/afterpulse_example_lake.jpg'
fig.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
pulse_group = dfs.groupby('pulseid')
df_pulse = pulse_group[['xatc', 'lat', 'lon']].mean()
df_pulse['ph_count'] = pulse_group['h'].count()
df_pulse['ph_count_smooth'] = np.array(df_pulse.ph_count.rolling(smooth_pulse,center=True,min_periods=1).mean())

pulse_group_top = df_top.groupby('pulseid')
df_pulse_top = pulse_group_top[['xatc', 'lat', 'lon']].mean()
df_pulse_top['ph_count'] = pulse_group_top['h'].count()
df_pulse_top['ph_count_smooth'] = np.array(df_pulse_top.ph_count.rolling(smooth_pulse,center=True,min_periods=1).mean())


In [ ]:
pulse_dfs[0]

In [ ]:
df_pulse

In [ ]:
mframe_group = df.groupby('mframe')
    df_mframe = mframe_group[['lat','lon', 'xatc', 'dt']].mean()
    df_mframe.drop(df_mframe.head(1).index,inplace=True)
    df_mframe.drop(df_mframe.tail(1).index,inplace=True)
    df_mframe['time'] = df_mframe['dt'].map(convert_time_to_string)
    df_mframe['xatc_min'] = mframe_group['xatc'].min()
    df_mframe['xatc_max'] = mframe_group['xatc'].max()
    df_mframe['n_phot'] = mframe_group['h'].count()

In [ ]:
# pulse_dfs[0].loc[pulse_dfs[0].iloc[:5].index]
df_pulses = pulse_dfs[0]
idx1 = df_pulses[df_pulses.ph_count > 0.99].index
idx2 = df_pulses[df_pulses.ph_count > 0.99].index
idx1.append(idx2)

In [ ]:
lk['photon_data']

In [ ]:
ax.get_xlim()